### Compute pauli z eigen value using QPE.

In [262]:
from qiskit import *
from qiskit.circuit.library import *
from qiskit.primitives import Sampler
import numpy as np 
from qiskit.extensions import *
from qiskit.algorithms import HamiltonianPhaseEstimation
from qiskit.quantum_info.operators import Operator, Pauli

A = [
        [1,0],
        [0,-1]
    ]


In [263]:
def convertBinaryToDecimal(str):
    str = list(str)
    o = 0 
    multiplication_factor = 2
    while str:
        if int(str.pop(0)): 
            o += 1/multiplication_factor
        multiplication_factor *= 2
    return o

In [264]:
import itertools
import functools as ft
from qiskit.quantum_info import SparsePauliOp

## ref : https://obliviateandsurrender.github.io/blogs/vqe.html
def decompose_ham_to_pauli(H):
    n = int(np.log2(len(H)))
    N = 2 ** n
    
    sI = np.eye(2, 2, dtype=complex)
    sX = np.array([[0, 1], [1, 0]], dtype=complex)
    sZ = np.array([[1, 0], [0,-1]], dtype=complex)
    sY = complex(0,-1)*np.matmul(sZ,sX)
    paulis = [sI, sX, sY, sZ]
    paulis_label = ['I', 'X', 'Y', 'Z']
    obs = []
    coeffs = []
    matrix = []
    for term in itertools.product(paulis, repeat=n):
        matrices = [pauli for pauli in term]
        coeff = np.trace(ft.reduce(np.kron, matrices) @ H) / N 
        coeff = np.real_if_close(coeff).item()
        # Hilbert-Schmidt-Product
        if not np.allclose(coeff, 0): 
            coeffs.append(coeff)
            obs.append(''.join([paulis_label[[i for i, x in enumerate(paulis)
            if np.all(x == t)][0]] for idx, t in enumerate(reversed(term))]))
            matrix.append(ft.reduce(np.kron, matrices))

    return obs, coeffs , matrix

o, amplitudes, _ = decompose_ham_to_pauli(A)
randHamiltonian = SparsePauliOp.from_list(list(zip(o, amplitudes))).to_operator()

print (randHamiltonian)

Q = QuantumCircuit(1)
Q.append(randHamiltonian, [0])

pe = PhaseEstimation(5, unitary=Q.to_gate())

state = QuantumCircuit(6, 5)
state.x(5)
state.barrier()
state.append(pe, [0,1,2,3,4,5])

for i in range(5):
    state.measure(i,i)
state.decompose().decompose().draw(fold=1000)

Operator([[ 1.+0.j,  0.+0.j],
          [ 0.+0.j, -1.+0.j]],
         input_dims=(2,), output_dims=(2,))


░ ┌───┐                                                                                           ┌──────────┐┌─┐            
q_0: ─────────────░─┤ H ├────────■──────────────────────────────────────────────────────────────────────────────────┤4         ├┤M├────────────
                  ░ ├───┤        │                                                                                  │          │└╥┘┌─┐         
q_1: ─────────────░─┤ H ├────────┼─────────────────■────────────────────────────────────────────────────────────────┤3         ├─╫─┤M├─────────
                  ░ ├───┤        │                 │                                                                │          │ ║ └╥┘┌─┐      
q_2: ─────────────░─┤ H ├────────┼─────────────────┼─────────────────■──────────────────────────────────────────────┤2 IQFT_dg ├─╫──╫─┤M├──────
                  ░ ├───┤        │                 │                 │                                              │          │ ║  ║ └╥┘┌─┐   
q_3: ─────────────░─┤ H ├────────┼─────────────────┼─────────────────┼─────────────────■────────────────────────────┤1         ├─╫──╫──╫─┤M├───
                  ░ ├───┤        │                 │                 │                 │                            │          │ ║  ║  ║ └╥┘┌─┐
q_4: ─────────────░─┤ H ├────────┼─────────────────┼─────────────────┼─────────────────┼──────────────────■─────────┤0         ├─╫──╫──╫──╫─┤M├
     ┌──────────┐ ░ └───┘┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌────────┴────────┐└──────────┘ ║  ║  ║  ║ └╥┘
q_5: ┤ U(π,0,π) ├─░──────┤ circuit-8471^1 ├┤ circuit-8471^2 ├┤ circuit-8471^4 ├┤ circuit-8471^8 ├┤ circuit-8471^16 ├─────────────╫──╫──╫──╫──╫─
     └──────────┘ ░      └────────────────┘└────────────────┘└────────────────┘└────────────────┘└─────────────────┘             ║  ║  ║  ║  ║ 
c: 5/════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩══╩═
                                                                                                                                 0  1  2  3  4

In [265]:
s = Sampler()
job = s.run(state)

print (job.result())

for key, value in job.result().quasi_dists[0].items():
    val = convertBinaryToDecimal(bin(key)[2:])
    if round(value, 3) > 0.1:
        print(key,val, "prob = ", round(value, 3), " Eigen value = ", np.round(np.cos(2*np.pi*val)+1j*np.sin(2*np.pi*val), 2))

SamplerResult(quasi_dists=[{1: 0.9999999999999984}], metadata=[{}])
1 0.5 prob =  1.0  Eigen value =  (-1+0j)
